# Program Manajemen Proyek IoT

Selamat datang di program **Manajemen Proyek IoT**. Program ini adalah sebuah sistem sederhana berbasis teks untuk mengelola data pengguna dan proyek-proyek IoT yang mereka tangani.

**Fitur Utama:**
* **Sistem Akun:** Pengguna dapat mendaftar (**Register**) dan masuk (**Login**) ke dalam sistem.
* **Manajemen Profil:** Setelah login, pengguna dapat melihat dan memperbarui data profil mereka.
* **Manajemen Proyek (CRUD):** Pengguna dapat mengelola daftar perangkat IoT yang terkait dengan akun mereka, meliputi:
    * **Create**: Menambahkan perangkat baru ke dalam proyek.
    * **Read**: Melihat daftar perangkat yang sudah terdaftar.
    * **Update**: Memperbarui status perangkat (misalnya dari 'Aktif' menjadi 'Maintenance').
    * **Delete**: Menghapus perangkat dari daftar.

In [1]:
data_akun = {
    'admin': 'admin123',
    'user01': 'pass01'
}

data_profil = {
    'admin': {
        'Nama': 'Admin Utama',
        'Jabatan': 'Project Manager Senior'
    },
    'user01': {
        'Nama': 'Budi Santoso',
        'Jabatan': 'IoT Engineer'
    }
}

menu_awal = ("Selamat Datang di Sistem Manajemen Proyek IoT", "Login", "Register", "Exit")
menu_utama = ("Menu Utama", "Kelola Proyek IoT", "Lihat Profil", "Update Profil", "Logout")
menu_crud = ("Menu Kelola Proyek IoT", "Tambah Perangkat IoT", "Lihat Daftar Perangkat", "Update Status Perangkat", "Hapus Perangkat", "Kembali ke Menu Utama")

data_proyek_iot = {
    'admin': [
        {'id': 'SN-001', 'tipe': 'Sensor Suhu', 'lokasi': 'Gudang A', 'status': 'Aktif'},
        {'id': 'AC-001', 'tipe': 'Aktuator Pendingin', 'lokasi': 'Gudang A', 'status': 'Aktif'}
    ],
    'user01': [
        {'id': 'SN-002', 'tipe': 'Sensor Kelembapan', 'lokasi': 'Ruang Server', 'status': 'Maintenance'}
    ]
}

### Alasan Pemilihan Tipe Sequence (Data Tambahan)
Struktur data untuk `data_proyek_iot` dipilih sebagai **Dictionary** yang berisi **List** dari **Dictionary** dengan alasan sebagai berikut:
1.  **Outer Dictionary**: Digunakan untuk memetakan data proyek langsung ke `ID` pengguna. Ini memastikan bahwa data setiap pengguna terisolasi dan dapat diakses dengan cepat (efisiensi O(1)), memenuhi syarat bahwa data yang dikelola terbatas pada akun yang sedang login.
2.  **List**: Di dalam setiap `ID` pengguna, sebuah `List` digunakan untuk menampung beberapa perangkat IoT. `List` dipilih karena jumlah perangkat bersifat dinamis (bisa ditambah atau dikurangi).
3.  **Inner Dictionary**: Setiap elemen di dalam `List` adalah sebuah `Dictionary` yang merepresentasikan satu perangkat IoT. `Dictionary` adalah pilihan ideal karena setiap perangkat memiliki atribut yang jelas (key-value pair) seperti 'id', 'tipe', dan 'status'.

In [2]:
def tampilkan_menu(menu_tuple):
    """Fungsi untuk menampilkan menu dari tuple."""
    print("\n" + "="*30)
    print(f"| {menu_tuple[0]:^26} |")
    print("="*30)

    for i, item in enumerate(menu_tuple[1:]):
        print(f"{i+1}. {item}")
    print("="*30)

def validasi_input_angka(prompt, rentang_maks):
    """Fungsi untuk memvalidasi input agar hanya menerima angka dalam rentang tertentu."""
    while True:
        pilihan = input(prompt)
        if pilihan.isdigit():
            pilihan_int = int(pilihan)
            if 1 <= pilihan_int <= rentang_maks:
                return pilihan_int
            else:
                print(f"Error: Pilihan harus antara 1 dan {rentang_maks}.")
        else:
            print("Error: Input harus berupa angka.")

In [3]:
def register():
    """Fungsi untuk mendaftarkan akun baru."""
    print("\n--- Halaman Register ---")
    while True:
        id_baru = input("Masukkan ID baru (misal: user02): ")
        if id_baru in data_akun:
            print("ID sudah terdaftar, silakan gunakan ID lain.")
        elif not id_baru:
             print("ID tidak boleh kosong.")
        else:
            break
    
    while True:
        password_baru = input("Masukkan password baru: ")
        if not password_baru:
            print("Password tidak boleh kosong.")
        else:
            break

    data_akun[id_baru] = password_baru
    nama = input("Masukkan Nama Lengkap Anda: ")
    jabatan = input("Masukkan Jabatan Anda (misal: IoT Specialist): ")
    data_profil[id_baru] = {'Nama': nama, 'Jabatan': jabatan}
    data_proyek_iot[id_baru] = []
    print("\nRegistrasi berhasil! Silakan login.")

def login():
    """Fungsi untuk login pengguna."""
    print("\n--- Halaman Login ---")
    percobaan = 0
    while percobaan < 3:
        id_login = input("Masukkan ID: ")
        password = input("Masukkan Password: ")
        if id_login in data_akun and data_akun[id_login] == password:
            print("\nLogin berhasil! Selamat datang,", data_profil[id_login]['Nama'])
            return id_login
        else:
            print("ID atau password salah.")
            percobaan += 1
    print("Gagal login 3 kali. Kembali ke menu awal.")
    return None

In [4]:
def lihat_perangkat(id_login):
    print("\n--- Daftar Perangkat IoT Anda ---")
    daftar = data_proyek_iot[id_login]
    if not daftar:
        print("Anda belum memiliki perangkat IoT.")
        return False
    else:
        print("-" * 60)
        print(f"{'No.':<4} | {'ID Perangkat':<15} | {'Tipe':<20} | {'Status':<15}")
        print("-" * 60)
        for i, perangkat in enumerate(daftar):
            print(f"{i+1:<4} | {perangkat['id']:<15} | {perangkat['tipe']:<20} | {perangkat['status']:<15}")
        print("-" * 60)
        return True

def kelola_proyek(id_login):
    """Fungsi sebagai hub untuk menu CRUD."""
    while True:
        tampilkan_menu(menu_crud)
        pilihan = validasi_input_angka("Pilih menu: ", len(menu_crud)-1)
        
        if pilihan == 1: # Tambah
            print("\n--- Tambah Perangkat IoT Baru ---")
            id_p = input("ID unik perangkat: ")
            tipe_p = input("Tipe Perangkat: ")
            lokasi_p = input("Lokasi Pemasangan: ")
            data_proyek_iot[id_login].append({'id': id_p, 'tipe': tipe_p, 'lokasi': lokasi_p, 'status': 'Aktif'})
            print(f"\nPerangkat '{id_p}' berhasil ditambahkan.")

        elif pilihan == 2: # Lihat
            lihat_perangkat(id_login)

        elif pilihan == 3: # Update
            if lihat_perangkat(id_login):
                daftar = data_proyek_iot[id_login]
                prompt = f"Pilih nomor perangkat (1-{len(daftar)}): "
                idx = validasi_input_angka(prompt, len(daftar)) - 1
                status_baru = input("Masukkan status baru: ")
                data_proyek_iot[id_login][idx]['status'] = status_baru
                print("\nStatus berhasil diperbarui.")

        elif pilihan == 4: # Hapus
            if lihat_perangkat(id_login):
                daftar = data_proyek_iot[id_login]
                prompt = f"Pilih nomor perangkat (1-{len(daftar)}): "
                idx = validasi_input_angka(prompt, len(daftar)) - 1
                dihapus = data_proyek_iot[id_login].pop(idx)
                print(f"\nPerangkat '{dihapus['id']}' berhasil dihapus.")

        elif pilihan == 5: # Kembali
            print("Kembali ke Menu Utama...")
            break

In [ ]:
def main():
    id_pengguna_login = None
    while True:
        if id_pengguna_login is None:
            tampilkan_menu(menu_awal)
            pilihan = validasi_input_angka("Pilih menu: ", len(menu_awal)-1)
            if pilihan == 1:
                id_pengguna_login = login()
            elif pilihan == 2:
                register()
            elif pilihan == 3:
                print("\nTerima kasih telah menggunakan program ini!")
                break
        else:
            tampilkan_menu(menu_utama)
            pilihan = validasi_input_angka("Pilih menu: ", len(menu_utama)-1)
            if pilihan == 1:
                kelola_proyek(id_pengguna_login)
            elif pilihan == 2:
                profil = data_profil[id_pengguna_login]
                print(f"\n--- Profil Anda ---\nID: {id_pengguna_login}\nNama: {profil['Nama']}\nJabatan: {profil['Jabatan']}")
            elif pilihan == 3:
                profil = data_profil[id_pengguna_login]
                nama_baru = input(f"Update Nama ({profil['Nama']}): ") or profil['Nama']
                jabatan_baru = input(f"Update Jabatan ({profil['Jabatan']}): ") or profil['Jabatan']
                data_profil[id_pengguna_login] = {'Nama': nama_baru, 'Jabatan': jabatan_baru}
                print("\nProfil berhasil diperbarui!")
            elif pilihan == 4:
                print("\nAnda telah logout.")
                id_pengguna_login = None

# Menjalankan program utama
if __name__ == "__main__":
    main()


| Selamat Datang di Sistem Manajemen Proyek IoT |
1. Login
2. Register
3. Exit
